# Vacancy Trajectory Analysis

Analyzing vacancy hopping in `VacHopPy` begins with the `Calculator()` factory function and the `Site` class.

---
## 1. The `Calculator()` Factory Function

The `Calculator()` function is the main entry point for `VacHopPy`'s analysis pipeline. It gathers all necessary inputs, performs initial setup, and returns a fully configured `CalculatorEnsemble` object, ready for analysis.

To get started, you call this function as follows:

```bash
calc_ensemble = Calculator(`path_traj`, `site`, `t_interval`)
```
### Key Arguments

* **`path_traj` (*str*)** 

    Path to the trajectory data. This can be either a single HDF5 trajectory file or a directory containing a bundle of HDF5 files. `VacHopPy` will automatically discover and process all valid files in the specified directory.

* **`site` (*Site*)**

    An instance of the `Site` class.

* **`t_interval` (*float, Optional*)**

    The time interval in **picoseconds (ps)** used for averaging atomic positions to determine site occupation. To distinguish true hopping events from thermal vibrations, `VacHopPy` averages the atomic coordinates and forces over each `t_interval`. This averaged quantities are then used to assign an atom to a specific lattice site. Each interval corresponds to a single analysis step. If this argument is not provided, `Calculator()` will automatically search for an optimal `t_interval` for the analysis.

### Return Value

Calling the `Calculator()` function returns an instance of the `CalculatorEnsemble` class, which holds all the processed data and methods required for subsequent analysis steps.

---
## 2. The `Site` Class

The `Site` class is used to define the structural backbone of your material. It identifies all **lattice sites** from a vacancy-free reference structure and determines the possible **hopping paths** between them.

You can create a `Site` object like this:

```bash
site = Site(`path_structure`, `symbol`)
```

### Key Arguments

* **`path_structure` (*str*)**

    Path to a structure file of the perfect, vacancy-free material. Any format supported by the **Atomic Simulation Environment (ASE)** is compatible.

* **`symbol` (*str*)**

    The chemical symbol of the diffusing species.

----

## 3. Usage Example

You can download source files to follow this tutorial from this (link)[(https://drive.google.com/file/d/1xBj3iP4eUInB2OKxCHstel4CTyXUDU9t/view?usp=sharing]


Navigate into the `Example6/` directory you downloaded. In this directory, you will find two files:

* **`TRAJ_O.h5`**

    This is the HDF5 file containing the MD trajectory data. The example system is **rutile TiO₂** containing **two oxygen vacancies**, simulated at **2100 K**.

* **`POSCAR_TiO2`**

    This file contains the crystal structure of the perfect, **vacancy-free** rutile rutile TiO₂ supercell.


In [17]:
import os
import numpy as np
from vachoppy.utils import show_traj
from vachoppy.core import Site, Calculator

In [2]:
path_traj, path_structure = 'TRAJ_O.h5', 'POSCAR_TiO2'

In [3]:
site = Site(path_structure, 'O')
site.summary()


  Structure File: POSCAR_TiO2
[Structure Information]
    - Structure Composition : Ti24 O48
    - Lattice Vectors (Ang) :
        [  9.29133,   0.00000,   0.00000]
        [  0.00000,   9.29133,   0.00000]
        [  0.00000,   0.00000,   8.90126]

[Hopping Path Information]
    - Diffusing Symbol   : O
    - Inequivalent Sites : 1 found
    - Inequivalent Paths : 3 found (with Rmax = 3.25 Å)

Name    Init Site    Final Site    a (Å)    z    Initial Coord (Frac)      Final Coord (Frac)
------  -----------  ------------  -------  ---  ------------------------  -------------------------
A1      site1        site1         2.5625   1    [0.0975, 0.4025, 0.1667]  [-0.0975, 0.5975, 0.1667]
A2      site1        site1         2.8031   8    [0.0975, 0.4025, 0.1667]  [0.3475, 0.3475, 0.0000]
A3      site1        site1         2.9671   2    [0.0975, 0.4025, 0.1667]  [0.0975, 0.4025, -0.1667]



In [4]:
show_traj(path_traj)

  Trajectory File: TRAJ_O.h5

[Simulation Parameters]
  - Atomic Symbol:      O
  - Number of Frames:   300000
  - Temperature:        2100.0 K
  - Time Step:          2.0 fs

[Composition]
  - Counts:             O: 46, Ti: 24
  - Total Atoms:        70

[Lattice Vectors (Ang)]
  [  9.29133,   0.00000,   0.00000]
  [  0.00000,   9.29133,   0.00000]
  [  0.00000,   0.00000,   8.90126]

[Stored Datasets]
  - positions:          Shape = (300000, 46, 3)
  - forces:             Shape = (300000, 46, 3)


In [5]:
calc_ensemble = Calculator(path_traj, site)
calc_ensemble.calculate()

                   Automatic t_interval Estimation
  Estimating from TRAJ_O.h5
             -> t_interval : 0.076 ps
        Adjusting t_interval to the nearest multiple of dt
    - dt                  : 0.0020 ps
    - Original t_interval : 0.0764 ps
    - Adjusted t_interval : 0.0760 ps (38 frames)


Analyze Trajectory: 100%|##############################| 1/1 [00:00<00:00, 1507.66it/s]



Analysis complete: 1 successful, 0 failed.
Execution Time: 2.386 seconds
Peak RAM Usage: 0.043 GB


In [6]:
for i, calc in enumerate(calc_ensemble.calculators):
    print(f"calculators[{i}] : {calc.path_traj}")
    
calc = calc_ensemble.calculators[0]

calculators[0] : /home/jty/Examples/Example6/TRAJ_O.h5


In [7]:
print(f"- Number of Vacancies                       : {calc.num_vacancies}")
print(f"- Time Step for the MD Simulation (dt)      : {calc.dt} fs")
print(f"- Time interval for Averaging  (t_interval) : {calc.t_interval} ps")
print(f"- Number of Frames in the MD Simulatoin     : {calc.num_frames} frames")
print(f"- Number of Frames per Step                 : {int(calc.t_interval * 1000 / calc.dt)} frames")
print(f"- Number of Steps for Analysis              : {calc.num_steps} steps")

- Number of Vacancies                       : 2
- Time Step for the MD Simulation (dt)      : 2.0 fs
- Time interval for Averaging  (t_interval) : 0.076 ps
- Number of Frames in the MD Simulatoin     : 299972 frames
- Number of Frames per Step                 : 38 frames
- Number of Steps for Analysis              : 7894 steps


In [8]:
calc.hopping_history[0][0]

{'site_init': 'site1',
 'site_final': 'site1',
 'distance': 2.8031139685542064,
 'z': 8,
 'coord_init': array([0.0975078 , 0.4024922 , 0.16666667]),
 'coord_final': array([0.3475078, 0.3475078, 0.       ]),
 'name': 'A2',
 'step': 157,
 'index_init': np.int64(0),
 'index_final': np.int16(25)}

In [9]:
calc.unwrapped_vacancy_trajectory_coord_cart[0]

array([[0.90597733, 3.73968856, 1.48354252],
       [3.73968856, 5.55164323, 4.45062755]])

### Interactive Vacancy Trajectory Plot 

In [10]:
calc.plot_vacancy_trajectory(vacancy_indices=[0, 1], unwrap=True)

### Animation : Site Occupation

In [11]:
calc.animate_occupation(step_init=5500, step_final=5525, dpi=100, fps=5)

Make Animation: 100%|##############################| 25/25 [00:00<00:00, 39598.79it/s]



Merging 25 snapshots into a GIF...
Successfully created 'occupation_video.gif'.

Execution Time: 17.430 seconds
Peak RAM Usage: 0.021 GB


In [12]:
for k, v in calc.transient_vacancy.items():
    if len(v) > 0:
        print(f"Step {k} ({k * calc.t_interval} ps) :  Site Index = {v}")
        print(f"  - Fractoinal Coord = {calc.lattice_sites[v][0]}")
        print(f"  - Cartesian Coord  = {calc.lattice_sites_cart[v][0]}")

Step 5516 (419.216 ps) :  Site Index = [20]
  - Fractoinal Coord = [0.9024922  0.0975078  0.83333333]
  - Cartesian Coord  = [8.38535446 0.90597733 7.41771258]


#### Interactive Animation

In [16]:
calc.animate_vacancy_trajectory(vacancy_indices=[0, 1], step_init=5500, step_final=5525, unwrap=False)

Make Animation: 100%|##############################| 26/26 [00:00<00:00, 77.40it/s]



'trajectory_video.html' created.

Execution Time: 0.626 seconds
Peak RAM Usage: 0.035 GB
